In [48]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde
from matplotlib import cm
from PIL import Image


np.random.seed(0)

# 生成数据
n = 10000
z = 2 * np.random.randn(n)
x = z + np.random.randn(n)
y = z - np.random.randn(n)

def pl(x,y,z, filename):
    # 计算整体 KDE
    xy = np.vstack([x, y])
    kde = gaussian_kde(xy)
    x_grid, y_grid = np.meshgrid(np.linspace(-9, 9, 100), np.linspace(-9, 9, 100))
    z_grid = kde(np.vstack([x_grid.ravel(), y_grid.ravel()])).reshape(x_grid.shape) * 2

    # 计算透明度映射
    z_grid_norm = (z_grid - np.min(z_grid) ) / (np.max(z_grid) - np.min(z_grid))
    facecolors = cm.get_cmap('RdYlGn')(z_grid_norm) # 获取颜色
    facecolors[..., 3] = z_grid_norm  # 透明度随高度变化

    # 计算 z 在特定区间的 KDE
    def compute_kde(z_range, cmap_name):
        mask = (z > z_range[0]) & (z < z_range[1])
        x_subset, y_subset = x[mask], y[mask]
        kde_subset = gaussian_kde(np.vstack([x_subset, y_subset]))
        z_grid_subset = kde_subset(np.vstack([x_grid.ravel(), y_grid.ravel()])).reshape(x_grid.shape) * 0.6
        z_grid_subset = np.clip(z_grid_subset, 0, 0.1)
        
        # 透明度映射
        z_norm = (z_grid_subset - np.min(z_grid_subset)) / (np.max(z_grid_subset) - np.min(z_grid_subset))
        facecolors_subset = cm.get_cmap(cmap_name)(z_norm)
        facecolors_subset[..., 3] = z_norm  # 透明度控制
        
        return z_grid_subset, facecolors_subset

    # 计算不同区间的 KDE 及透明度
    z_grid1, facecolors1 = compute_kde((1, 1.5), 'Reds')
    z_grid2, facecolors2 = compute_kde((-3.5, -2), 'Blues')

    # 创建 3D 图
    fig = plt.figure(figsize=(8, 6), dpi=500)
    ax = fig.add_subplot(111, projection='3d')

    # 隐藏坐标轴
    ax.set_axis_off()
    ax.grid(False)
    fig.patch.set_alpha(0)

    ax.plot_surface(x_grid, y_grid, np.zeros_like(x_grid), cmap='cividis', alpha=0.07)


    # 绘制等高面（透明度随 z 高度变化）
    ax.plot_surface(x_grid, y_grid, z_grid, facecolors=facecolors, rstride=1, cstride=1, linewidth=0.0)
    ax.plot_surface(x_grid, y_grid, z_grid2, facecolors=facecolors2, rstride=1, cstride=1, linewidth=0.08)
    #ax.plot_surface(x_grid, y_grid, z_grid1, facecolors=facecolors1, rstride=1, cstride=1)
    

    # 绘制散点
    sc = ax.scatter(x, y, np.zeros_like(x), c=z, cmap='coolwarm', marker='o', alpha=1, s=0.5)

    # 设置 z 轴范围
    ax.set_zlim(0, 0.45)

    # 调整视角
    ax.view_init(elev=20, azim=180)

    # 显示图形
    #plt.show()

    plt.savefig(filename+'.svg', bbox_inches='tight', pad_inches=-0.85, format="svg")
    plt.close(fig)  # 关闭图像，释放内存


pl(x,y,z, '0')
pl(x[200:400], y[200:400], z[200:400], '1')
pl(x[400:600], y[400:600], z[400:600], '2')
pl(x[600:800], y[600:800], z[600:800], '3')